### Alternating Least Squares for Matrix Factorization

In [1]:
import numpy as np
A = np.array([2,1,0,1,0,5])
B = np.array([4,0,2,0,0,0])
C = np.array([3,3,0,5,1,0])
D = np.array([0,0,5,0,1,2])
M = np.vstack((A,B,C,D))
M

array([[2, 1, 0, 1, 0, 5],
       [4, 0, 2, 0, 0, 0],
       [3, 3, 0, 5, 1, 0],
       [0, 0, 5, 0, 1, 2]])

In [2]:
U = np.random.choice(5, (4,4))
V = np.random.choice(5, (4,6))
print('\nU (4x4):\n{}\n\n\nV (4x6):\n{}'.format(U,V))


U (4x4):
[[2 2 1 2]
 [0 2 0 2]
 [1 3 1 4]
 [0 2 3 4]]


V (4x6):
[[2 1 3 2 4 0]
 [1 0 1 0 1 2]
 [1 1 3 3 1 1]
 [2 3 4 0 3 3]]


In [3]:
U.T.dot(V)

array([[ 5,  3,  9,  7,  9,  1],
       [13, 11, 25, 13, 19, 13],
       [ 9, 11, 18,  5, 14, 10],
       [18, 18, 36, 16, 26, 20]])

In [4]:
from numpy.linalg import inv, norm
np.set_printoptions(precision=2)
lbda = 0.2
m, n = len(M), len(M[0])
d = min(m,n)

In [5]:
v_ = lambda j: V[:,j]
u_ = lambda i: U[:,i]
vect_sq = lambda vect: vect.reshape(-1,1).dot(vect.reshape(1,-1))

In [6]:
omega = lambda coords: [(coords[0][t], coords[1][t]) for t in range(len(coords[0]))]

The loss that we are trying to minimize over U and V is given by sum of squared error between M and M_hat plus regularization error, given below:
\begin{equation*}
\min_{U,V} \left( \sum_{M_{ij} \in observed}^{} (M_{ij} - u_i^Tv_j)^2 \right) + \lambda\left( \sum_{i=1}^m \Vert u_i \Vert^2 + \sum_{j=1}^n \Vert v_j \Vert^2 \right)
\end{equation*}

In [7]:
def loss():
    sq_err = sum([(M[i][j] - u_(i).dot(v_(j)))**2 for (i,j) in omega(np.nonzero(M))])
    reg_loss = lbda*(sum([norm(u_(i))**2 for i in range(m)]) + sum([norm(v_(j))**2 for j in range(n)]))
    return sq_err + reg_loss

The closed form solution to obtain new estimates of U and V are obtained by differentiating the loss function w.r.t to the target optimization matrix each time
\begin{equation*}
u_i = \left( \sum_{j \in \Omega(i)}^{} (v_jv_j^T + \lambda I_k) \right)^{-1}\left( \sum_{j \in \Omega(i)} M_{ij}v_j \right)\\
v_j = \left( \sum_{i \in \Omega(j)}^{} (u_iu_i^T + \lambda I_k) \right)^{-1}\left( \sum_{i \in \Omega(j)} M_{ij}u_i \right)
\end{equation*}

In [8]:
def opt_U():
    U_hat = np.zeros(U.shape)
    for i in range(m):
        omega_i = [j for j in range(n) if M[i][j]!=0]
        A = inv(sum([(vect_sq(v_(j)) + lbda*np.eye(d)) for j in omega_i]))
        B = sum([(M[i][j]*v_(j)) for j in omega_i]).reshape(-1,1)
        U_hat[:,i] = np.squeeze(A.dot(B))
    return U_hat

In [9]:
def opt_V():
    V_hat = np.zeros(V.shape)
    for j in range(n):
        omega_j = [i for i in range(m) if M[i][j]!=0]
        A = inv(sum([(vect_sq(u_(i)) + lbda*np.eye(d)) for i in omega_j]))
        B = sum([(M[i][j]*u_(i)) for i in omega_j]).reshape(-1,1)
        V_hat[:,j] = np.squeeze(A.dot(B))
    return V_hat

In [10]:
for i in range(10):
    U = opt_U()
    print('Loss - {}'.format(loss()))
    V = opt_V()
    print('Loss - {}'.format(loss()))

Loss - 28.00437839609033
Loss - 16.29821818254604
Loss - 11.254134694362751
Loss - 10.977922610312213
Loss - 9.547057643085802
Loss - 9.699477240448566
Loss - 8.944564187929611
Loss - 9.123856879328283
Loss - 8.68574671150716
Loss - 8.846631754743626
Loss - 8.574814767931835
Loss - 8.701158574356379
Loss - 8.52650521237451
Loss - 8.619859037922362
Loss - 8.505285657788049
Loss - 8.57250234779038
Loss - 8.496156533603205
Loss - 8.544166671219466
Loss - 8.492603341915531
Loss - 8.52697498642881


In [11]:
M

array([[2, 1, 0, 1, 0, 5],
       [4, 0, 2, 0, 0, 0],
       [3, 3, 0, 5, 1, 0],
       [0, 0, 5, 0, 1, 2]])

In [12]:
M_hat = U.T.dot(V)
M_hat

array([[1.97, 0.97, 1.61, 1.04, 0.34, 4.44],
       [3.52, 2.13, 2.01, 3.28, 0.6 , 2.96],
       [2.92, 2.73, 3.91, 4.53, 0.95, 1.68],
       [1.83, 2.25, 4.5 , 3.65, 0.94, 1.99]])

In [13]:
users = { 0: 'A',
          1: 'B',
          2: 'C',
          3: 'D'
        }

In [14]:
print('\nUsers A,B,C,D have choices 1,2,3,4,5,6\n')
for i in range(m):
    print('User suggestions - {} => {}'.format(users[i], 1+np.argmax((M_hat-M)[i])))


Users A,B,C,D have choices 1,2,3,4,5,6

User suggestions - A => 3
User suggestions - B => 4
User suggestions - C => 3
User suggestions - D => 4
